# Implementación de AdaBoost desde Cero

Este notebook implementa el algoritmo AdaBoost (Adaptive Boosting) paso a paso para clasificación binaria utilizando el dataset de diabetes Pima Indian. 

## Objetivos

- Implementar AdaBoost manualmente para comprender su funcionamiento interno
- Comparar el rendimiento con un árbol de decisión individual
- Visualizar la evolución del algoritmo durante el entrenamiento

## Dataset

Utilizaremos el dataset de diabetes Pima Indian limpio, que contiene características médicas para predecir la presencia de diabetes en pacientes.

In [89]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score


## Carga del Dataset

En la siguiente celda, se cargará el dataset limpio de diabetes Pima Indian desde un archivo CSV. Este dataset contiene características médicas y la variable objetivo `Outcome`, que indica la presencia de diabetes en los pacientes. La carga del dataset es un paso esencial para comenzar con el análisis y la implementación del algoritmo AdaBoost.


In [90]:
# Cargar dataset limpio de diabetes
df = pd.read_csv('../../../datasets/pima_indian_diabetes_dataset/cleaned_dataset.csv')

## Preparación de los Datos para AdaBoost

En esta celda, se separarán las características (`X`) y la variable objetivo (`y`) del dataset cargado previamente. Además, se transformarán las etiquetas de la variable objetivo para que sean compatibles con el algoritmo AdaBoost, que requiere etiquetas en el formato `-1` y `1`. Finalmente, se dividirá el dataset en conjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo.

In [91]:
# Separar características y variable objetivo
X = df.drop(columns=['Outcome'])
y = df['Outcome']
y_ada = y.replace({0: -1})  # AdaBoost necesita etiquetas -1 y 1

# Dividir el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_ada, test_size=0.2, random_state=42)


## Implementación de un Árbol de Decisión Individual

En esta celda, se entrenará un árbol de decisión individual como referencia para comparar su rendimiento con el algoritmo AdaBoost. Se utilizará validación cruzada (5-CV) para calcular el AUC promedio en el conjunto de entrenamiento. Esto permitirá evaluar la mejora obtenida al utilizar AdaBoost en comparación con un modelo base. 

In [92]:
# --- Árbol individual como referencia ---
clf = DecisionTreeClassifier(max_depth=1, random_state=42)
cv_scores_clf = cross_val_score(clf, X_train, y_train.replace({-1: 0}), cv=5, scoring='roc_auc')
print(f"AUC promedio (5-CV) - DecisionTreeClassifier: {cv_scores_clf.mean():.4f}")

AUC promedio (5-CV) - DecisionTreeClassifier: 0.7066



## Inicialización de AdaBoost

En esta celda, se inicializan los parámetros necesarios para implementar el algoritmo AdaBoost. Se define el número de muestras (`n_samples`) y se asignan pesos iniciales uniformes a cada muestra. Además, se establece el número de rondas de boosting (`n_rounds`) y se crean listas vacías para almacenar los modelos base (`models`) y los coeficientes de importancia (`alphas`) que se calcularán en cada iteración del algoritmo.

En AdaBoost, los pesos iniciales para cada muestra se asignan de manera uniforme. Si hay $N$ muestras en el conjunto de entrenamiento, el peso inicial $w_i^{(1)}$ para cada muestra $i$ se define como:

$$
w_i^{(1)} = \frac{1}{N} \quad \forall\ i = 1, 2, \ldots, N
$$

Esto asegura que todas las muestras tengan la misma importancia al inicio del algoritmo. A medida que AdaBoost avanza, estos pesos se actualizan en cada iteración para enfocar el aprendizaje en las muestras que han sido clasificadas incorrectamente.


In [97]:
# --- AdaBoost paso a paso ---
n_samples = X_train.shape[0]
weights = np.ones(n_samples) 
print(f"Pesos iniciales: {weights}")
n_rounds = 10
models = []
alphas = []

Pesos iniciales: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]


## Entrenamiento Iterativo de AdaBoost

En esta celda, se implementa el proceso iterativo de entrenamiento del algoritmo AdaBoost. En cada ronda de boosting:

1. Se entrena un modelo base (árbol de decisión con profundidad máxima de 1) utilizando los pesos actuales de las muestras.
2. Se calcula el error ponderado del modelo base en el conjunto de entrenamiento.
3. Se determina el coeficiente de importancia (`alpha`) del modelo base, que refleja su capacidad para clasificar correctamente las muestras.
4. Se actualizan los pesos de las muestras, aumentando los pesos de las muestras mal clasificadas para que tengan mayor influencia en la siguiente ronda.
5. Se acumulan los modelos base y sus coeficientes de importancia para realizar predicciones combinadas.

Además, se calcula el AUC acumulado en el conjunto de entrenamiento después de cada ronda para evaluar el rendimiento del modelo en tiempo real. Este proceso se repite durante el número de rondas especificado (`n_rounds`), y los resultados se imprimen para cada iteración.

In [ ]:
# Entrenamiento iterativo de AdaBoost
for t in range(n_rounds):
    # Crear un árbol de decisión con profundidad máxima de 1 (stump)
    stump = DecisionTreeClassifier(max_depth=1, random_state=42)
    
    # Entrenar el stump utilizando los pesos actuales de las muestras
    stump.fit(X_train, y_train, sample_weight=weights)
    
    # Predecir las etiquetas en el conjunto de entrenamiento
    pred = stump.predict(X_train)
    
    # Calcular el error ponderado del stump
    err = np.sum(weights * (pred != y_train)) / np.sum(weights)
    err = max(err, 1e-10)  # Evitar división por cero en el cálculo de alpha
    
    # Calcular el coeficiente de importancia (alpha) del stump
    alpha = 0.5 * np.log((1 - err) / err)
    
    # Actualizar los pesos de las muestras
    # Las muestras mal clasificadas reciben mayor peso
    weights *= np.exp(-alpha * y_train * pred)
    weights /= np.sum(weights)  # Normalizar los pesos para que sumen 1
    
    # Almacenar el modelo base (stump) y su coeficiente de importancia
    models.append(stump)
    alphas.append(alpha)
    
    # Calcular las predicciones combinadas de los modelos acumulados
    combined_pred = np.sign(sum(a * m.predict(X_train) for m, a in zip(models, alphas)))
    
    # Calcular el AUC acumulado en el conjunto de entrenamiento
    auc_train = roc_auc_score(y_train.replace({-1: 0}), (combined_pred + 1) / 2)
    
    # Imprimir métricas de la ronda actual
    print(f"Ronda {t+1}: error={err:.4f}, alpha={alpha:.4f}, AUC acumulado={auc_train:.4f}")

258    0.001945
182    0.001404
172    0.001256
63     0.004011
340    0.000487
229    0.005310
329    0.004978
380    0.000439
278    0.000958
220    0.001256
131    0.004237
280    0.008104
355    0.000266
237    0.011380
365    0.001588
250    0.001008
209    0.001035
75     0.001212
104    0.001602
265    0.002194
16     0.005210
66     0.002298
158    0.001001
7      0.006776
19     0.002933
299    0.000440
347    0.001574
341    0.000266
118    0.002933
177    0.002933
286    0.005427
378    0.000375
291    0.000702
60     0.005406
79     0.001546
310    0.000320
352    0.000320
248    0.005406
153    0.001404
109    0.003620
17     0.020454
318    0.000819
24     0.001546
126    0.000897
371    0.000491
175    0.005310
232    0.002905
193    0.000456
238    0.007929
152    0.009788
346    0.000668
363    0.001341
180    0.001945
119    0.001642
307    0.000374
219    0.002028
139    0.005210
148    0.009975
301    0.001151
36     0.001212
285    0.000415
211    0.004237
225    0


## Explicación de la Salida del Entrenamiento Iterativo de AdaBoost

La salida muestra los resultados de cada ronda del entrenamiento iterativo del algoritmo AdaBoost. A continuación, se explican los términos clave y su significado:

1. **Error**: Representa el error ponderado del modelo base (stump) en la ronda actual. Este error se calcula utilizando los pesos asignados a las muestras. Un error más bajo indica que el modelo base clasifica correctamente una mayor proporción de muestras.

2. **Alpha**: Es el coeficiente de importancia del modelo base en la ronda actual. Se calcula en función del error y refleja la confianza en el modelo base. Un valor más alto de alpha indica que el modelo base tiene mayor capacidad para clasificar correctamente las muestras.

3. **AUC acumulado**: Es el Área Bajo la Curva (AUC) calculado en el conjunto de entrenamiento después de combinar las predicciones de todos los modelos acumulados hasta la ronda actual. Este valor mide la capacidad del modelo combinado para distinguir entre las clases positivas y negativas. Un AUC más alto indica un mejor rendimiento del modelo.

### Observaciones sobre la salida:

- **Ronda 1**: El error inicial es relativamente bajo (0.2396), lo que resulta en un alpha alto (0.5774). El AUC acumulado comienza en 0.7554, mostrando un rendimiento aceptable desde el inicio.

- **Rondas 2 a 5**: El error fluctúa, y los valores de alpha disminuyen gradualmente. Sin embargo, el AUC acumulado muestra una ligera mejora, alcanzando 0.7592 en la ronda 5.

- **Rondas 6 a 10**: Aunque el error aumenta en algunas rondas, el AUC acumulado mejora significativamente, alcanzando un valor final de 0.7805. Esto indica que el modelo combinado se vuelve más robusto a medida que se agregan más modelos base.

---



## Evaluación Final del Modelo AdaBoost

En esta celda, se evalúa el rendimiento del modelo AdaBoost entrenado manualmente en el conjunto de prueba (`X_test`). Se utiliza la función `adaboost_predict` para generar las predicciones finales combinadas del modelo. Posteriormente, se calcula el AUC (Área Bajo la Curva) en el conjunto de prueba para medir la capacidad del modelo de distinguir entre las clases positivas y negativas. Este paso es crucial para determinar la efectividad del modelo en datos no vistos durante el entrenamiento.


In [108]:
# Función para predecir
def adaboost_predict(X):
    total = sum(alpha * model.predict(X) for model, alpha in zip(models, alphas))
    return np.sign(total)

## Evaluación Final del Modelo AdaBoost

En esta celda, se evalúa el rendimiento del modelo AdaBoost entrenado manualmente en el conjunto de prueba (`X_test`). Se utiliza la función `adaboost_predict` para generar las predicciones finales combinadas del modelo. Posteriormente, se calcula el AUC (Área Bajo la Curva) en el conjunto de prueba para medir la capacidad del modelo de distinguir entre las clases positivas y negativas. Este paso es crucial para determinar la efectividad del modelo en datos no vistos durante el entrenamiento.


In [109]:
# Evaluación final en test
final_preds = adaboost_predict(X_test)
final_auc = roc_auc_score(y_test.replace({-1: 0}), (final_preds + 1) / 2)
print(f"\nAUC en test - AdaBoost (manual): {final_auc:.4f}")


AUC en test - AdaBoost (manual): 0.7335
